## Raw VCF files conversion to ADAM format (avro parquet files)

In [ ]:
import org.apache.hadoop.fs.{FileSystem, Path}
//import org.bdgenomics.adam.converters.{ VCFLine, VCFLineConverter, VCFLineParser }
import org.bdgenomics.formats.avro.{Genotype}
import org.bdgenomics.adam.models.VariantContext
import org.bdgenomics.adam.rdd.ADAMContext._
import org.bdgenomics.adam.rdd.ADAMContext
import org.apache.spark.rdd.RDD

import org.apache.spark.sql.DataFrame

In [ ]:
val vcf = "/tmp/6-sample.vcf"
val localAdam = vcf+"6"+".adam"

In [ ]:
val gts = sparkContext.loadGenotypes(localAdam)

In [ ]:
gts.rdd.map(g =>  g.getSampleId).take(3)

In [ ]:
val sampleCount = gts.rdd.map(_.getSampleId.toString.hashCode).distinct.count
s"#Samples: $sampleCount"

In [ ]:
import scala.collection.JavaConverters._

val allal = gts.rdd.map{ g => (g.getVariant.getReferenceAllele, g.getAlleles.asScala.map(_.toString)) }.toDF.cache()
allal.toDF("var","coucou").take(10)

In [ ]:
import scala.collection.JavaConverters._
val genotypesDF = gts.rdd.map{ genotype => {
  def asDouble(g:Genotype):Double = g.getAlleles.asScala.map(_.toString).count(_ != "REF")
  def variantId(g:Genotype):String = {
    val name = g.getVariant.contigName
    val start = g.getVariant.getStart
    val end = g.getVariant.getEnd
    s"$name:$start:$end"
  } 
  (genotype.getSampleId.toString,
   variantId(genotype),
   asDouble(genotype)
   )
}
}.toDF("sampleId","variantId","genotype")
   

In [ ]:
genotypesDF.schema

In [ ]:
genotypesDF.write.parquet(vcf+"6"+".parquet")

In [ ]:
:sh ls -lh ${vcf}6.parquet

In [ ]:
genotypesDF.select("genotype").distinct